# DATA GATHERING AND PRE-PROCESSIG YFINANCE DATA

## Pre-processing and storing NASDAQ rep data
The Nasdaq Stock Market is an American stock exchange based in New York City. It is the most active stock trading venue in the US by volume, and ranked second on the list of stock exchanges by market capitalization of shares traded, behind the New York Stock Exchange. 

### Subset of Symbols on NASDAQ
__Link:__ https://www.nasdaq.com/market-activity/stocks/screener

In [2]:
import pandas as pd

In [3]:
df_nasdaq = pd.read_csv("nasdaq_data.csv")

##### Dropping rows without IPO Years

In [4]:
df_nasdaq = df_nasdaq.dropna(subset=['IPO Year'])

In [5]:
df_nasdaq.reset_index(inplace=True,drop=True)

__Creating new column Max Years__

In [6]:
df_nasdaq['Max Years'] = 2023 - df_nasdaq["IPO Year"]

In [7]:
df_nasdaq.sample(5)

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,Max Years
1888,GTAC,Global Technology Acquisition Corp. I Class A ...,$10.42,0.0000,0.00%,0.000000e+00,United States,2021.0,1,Finance,Blank Checks,2.0
4131,TTD,The Trade Desk Inc. Class A Common Stock,$62.70,1.2600,2.051%,3.060238e+10,United States,2016.0,2431065,Technology,EDP Services,7.0
3907,STSA,Satsuma Pharmaceuticals Inc. Common Stock,$1.07,0.0200,1.905%,3.547317e+07,United States,2019.0,176488,Health Care,Biotechnology: Pharmaceutical Preparations,4.0
441,AVGR,Avinger Inc. Common Stock,$0.4723,0.0184,4.054%,4.068632e+06,United States,2015.0,125862,Health Care,Medical/Dental Instruments,8.0
2686,MLCO,Melco Resorts & Entertainment Limited American...,$13.01,0.2800,2.20%,5.766375e+09,Hong Kong,2006.0,2617939,Consumer Discretionary,Hotels/Resorts,17.0


In [8]:
df_nasdaq[["Symbol","Name","Max Years"]].sample(5)

,Symbol,Name,Max Years
3410,QNCX,Quince Therapeutics Inc. Common Stock,4.0
172,AIZN,Assurant Inc. 5.25% Subordinated Notes due 2061,3.0
112,AFCG,AFC Gamma Inc. Common Stock,2.0
1625,FOXF,Fox Factory Holding Corp. Common Stock,10.0
480,BAH,Booz Allen Hamilton Holding Corporation Common...,13.0


In [9]:
meta_arr = df_nasdaq[["Symbol","Name","Max Years"]].values

In [10]:
len(meta_arr)

4524

## Creating a collection of metadata with all the info of the companies

In [11]:
import yfinance as yf
from pymongo import MongoClient

# Define the MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client['finance']
collection = db['tickers_meta']

In [10]:
def query_ticker(symbol):
    # Query the ticker data using yfinance
    ticker_data = yf.Ticker(symbol).info
    return ticker_data

def insert_ticker_data(data):
    # Insert the ticker data into the MongoDB collection
    collection.insert_one(data)

In [ ]:
for symbol in meta_arr:
    
    symbol_data = {'Symbol': symbol[0],
                   'Name': symbol[1],
                   'Max Years': symbol[2],
                   'Data': query_ticker(symbol[0])}
    
    insert_ticker_data(symbol_data)

## Extracting the common features amongs all collections for easier comparison

In [2]:
match_query = { "Data": { "$exists": True } }
document_count = db.tickers_meta.count_documents(match_query)

pipeline = [
    { "$match": match_query },
    { "$project": { "dataFields": { "$objectToArray": "$Data" } } },
    { "$unwind": "$dataFields" },
    { "$group": { "_id": "$dataFields.k", "count": { "$sum": 1 } } },
    { "$match": { "count": document_count } },
    { "$project": { "_id": 0, "attribute": "$_id" } }
]

results = db.tickers_meta.aggregate(pipeline)
for result in results:
    print(result)


{'attribute': 'symbol'}
{'attribute': 'previousClose'}
{'attribute': 'regularMarketVolume'}
{'attribute': 'gmtOffSetMilliseconds'}
{'attribute': 'volume'}
{'attribute': 'uuid'}
{'attribute': 'regularMarketDayHigh'}
{'attribute': 'timeZoneShortName'}
{'attribute': 'dayLow'}
{'attribute': 'regularMarketDayLow'}
{'attribute': 'timeZoneFullName'}
{'attribute': 'quoteType'}
{'attribute': 'regularMarketPreviousClose'}
{'attribute': 'fiftyTwoWeekLow'}
{'attribute': 'maxAge'}
{'attribute': 'priceHint'}
{'attribute': 'trailingPegRatio'}
{'attribute': 'exchange'}
{'attribute': 'underlyingSymbol'}
{'attribute': 'dayHigh'}
{'attribute': 'fiftyTwoWeekHigh'}


## Deleting Duplicate values in MongoDB

In [51]:
# Delete documents with duplicate Name fields
duplicate_names = collection.aggregate([
    {'$group': {'_id': '$Name', 'count': {'$sum': 1}}},
    {'$match': {'count': {'$gt': 1}}}
])
for document in duplicate_names:
    collection.delete_many({'Name': document['_id']})

In [15]:
df_nasdaq.columns

Index(['Symbol', 'Name', 'Last Sale', 'Net Change', '% Change', 'Market Cap',
       'Country', 'IPO Year', 'Volume', 'Sector', 'Industry', 'Max Years'],
      dtype='object')

## Adding more data (update_one) into each document of tickers_meta

In [47]:
tickers_meta_document = collection.find()

count = 0 
for document in tickers_meta_document:
    
    if document['Symbol'] in symbols_dataset:
        
        symbol = document['Symbol']
        symbol_subset = df_nasdaq[df_nasdaq['Symbol']==symbol]
        
        data = symbol_subset[['Net Change', 
                       '% Change', 
                       'Market Cap',
                       'Country', 
                       'IPO Year', 
                       'Volume', 
                       'Sector', 
                       'Industry']].values[0]
    
        new_fields = {'Net Change': data[0], 
                       '% Change': data[1], 
                       'Market Cap': data[2],
                       'Country': data[3], 
                       'IPO Year': data[4], 
                       'Volume': data[5], 
                       'Sector': data[6], 
                       'Industry': data[7]}
        


        # Update document with new fields
        result = collection.update_one({"Symbol": symbol}, {"$set": new_fields})
        
        
    else:
        continue

# Cross-Cehcking whether the Tickers (Stocks) exist on yfinance API

In [12]:
df_yfin_tick = pd.read_excel("yahoo_finance_ticker_symbol.xlsx")

In [64]:
tickers_yfin = list(df_yfin_tick.Ticker.values)
df_yfin_tick.sample(5)

,Ticker,Name,Exchange,Category Name,Country
55668,CHEOF,Cochlear Limited,PNK,NaN,USA
4251,ZBRA,Zebra Technologies Corporation,NMS,Diversified Machinery,USA
76826,SFI.F,ShiFang Holding Limited,FRA,NaN,France
67642,FGT.DU,FENGHUA SOLETECH AG,DUS,NaN,Germany
91680,ORLY.MX,"O'Reilly Automotive, Inc.",MEX,"Specialty Retail, Other",Mexico


In [65]:
len(df_yfin_tick)

106328

In [66]:
tickers_meta_symbols = collection.find()

symbols_meta = []
for elements in tickers_meta_symbols:
    symbols_meta.append(elements['Symbol'])


In [68]:
symbols_union = []
for x in symbols_meta:
    if x in tickers_yfin:
        symbols_union.append(x)

In [69]:
len(symbols_union)

1758

__Inserting elements from the old collection to the new collection__

In [72]:
new_collection = db['tickers_meta_ref']

for symb_i in symbols_union:
    document = collection.find({"Symbol":symb_i})
    for doc in document:
        new_collection.insert_one(doc)